In [2]:
import pandas as pd
from pprint import pprint
import glob
import pandas as pd
import random
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config import event_types
random.seed(42)
pd.set_option('display.max_rows', None)
# jupyter nbconvert --to script 04b_combine_results_find_disagreement.ipynb
# sudo kill -9 $(nvidia-smi | awk 'NR>8 {print $5}' | grep -E '^[0-9]+$')
version = 2
from config import event_types


   

def prepare_df(df, type="biolord"):
    global event_types
    df = df.copy()
    print("Loaded file len: ",len(df))
    df['Sent_ID'] = df['Events'].apply(lambda x: [f"{i:04d}" for i in range(len(x))])
    df = df.explode(["Sent_ID","Events"])
    print("After exploding file len: ",len(df))
    df['UID'] = df['SUBJECT_ID'].astype(str) + "_" + df['ROW_ID'].astype(str) + "_" + df['Sent_ID'].astype(str)
    df = df.dropna(subset="Events")
    df['Event_Name'] = df['Events'].apply(lambda x: x['event'])
    df['Sentence'] = df['Events'].apply(lambda x: x['text'])
    df['Time'] = df['Events'].apply(lambda x: x['event_detection_time'])
    
    if type == "dictionary":
        df['Keyword'] = df['Events'].apply(lambda x: x['keyword'])
        df['Lemma'] = df['Events'].apply(lambda x: x['lemma'])
        df['Keyword_Position'] = df['Events'].apply(lambda x: x['keyword_position'])
        
    if type == "biolord":
        df['Similarity'] = df['Events'].apply(lambda x: x['similarity'])
        df['Similarity'] = df['Similarity'].apply(lambda x: {k:v for (k,v) in x.items() if k!="Alert And Oriented"})
        for ET in event_types:
            df[f"{ET}_similarity"] = df['Similarity'].apply(lambda x:x[ET])
            
    return df
df_dictionary = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_dictionary_v{version}/combined.pkl"),type="dictionary")
# df_biolord = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_biolord_v{version}/combined.pkl"), type = "biolord")






Loaded file len:  14483
After exploding file len:  322079


In [3]:
df_dictionary.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'DOCUMENT',
       'AGE', 'LOS_DAYS', 'IS_ALIVE', 'DOCUMENT_LOWER', 'NUM_NOTES',
       'MAX_NOTES_PER_DAY', 'MAX_NOTES_PER_CHARTTIME', 'all_dates_present',
       'Sentences', 'Sentences_Raw', 'Sentences_Cleaned', 'Events', 'Sent_ID',
       'UID', 'Event_Name', 'Sentence', 'Time', 'Keyword', 'Lemma',
       'Keyword_Position'],
      dtype='object')

In [4]:
df_dictionary_exploded = df_dictionary.explode(['Keyword','Lemma','Event_Name','Keyword_Position'])
df_dictionary_exploded['KUID'] = df_dictionary_exploded['UID'] + "_" + df_dictionary_exploded['Keyword_Position'].astype(str)


In [5]:
df_dictionary.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,Sentences_Cleaned,Events,Sent_ID,UID,Event_Name,Sentence,Time,Keyword,Lemma,Keyword_Position
0,333811,26851,169435,2139-07-13,2139-07-13 10:19:00,2139-07-13 10:20:01,Nursing,Nursing Progress Note,17745.0,NaN,...,"[subarachnoid hemorrhage (sah), assessment: s...","{'text': 'subarachnoid hemorrhage (sah)', 'eve...",0000,26851_333811_0000,[Unknown],subarachnoid hemorrhage (sah),0.056522,[],[],[]
0,333811,26851,169435,2139-07-13,2139-07-13 10:19:00,2139-07-13 10:20:01,Nursing,Nursing Progress Note,17745.0,NaN,...,"[subarachnoid hemorrhage (sah), assessment: s...",{'text': 'assessment: stable neurologically i...,0001,26851_333811_0001,[Unknown],assessment: stable neurologically intact,0.002603,[],[],[]
0,333811,26851,169435,2139-07-13,2139-07-13 10:19:00,2139-07-13 10:20:01,Nursing,Nursing Progress Note,17745.0,NaN,...,"[subarachnoid hemorrhage (sah), assessment: s...","{'text': 'action: zofran, raglan, mso4 given ...",0002,26851_333811_0002,[Unknown],"action: zofran, raglan, mso4 given for modera...",0.002590,[],[],[]
0,333811,26851,169435,2139-07-13,2139-07-13 10:19:00,2139-07-13 10:20:01,Nursing,Nursing Progress Note,17745.0,NaN,...,"[subarachnoid hemorrhage (sah), assessment: s...","{'text': 'response: improved per patient', 'e...",0003,26851_333811_0003,[Unknown],response: improved per patient,0.002463,[],[],[]
0,333811,26851,169435,2139-07-13,2139-07-13 10:19:00,2139-07-13 10:20:01,Nursing,Nursing Progress Note,17745.0,NaN,...,"[subarachnoid hemorrhage (sah), assessment: s...","{'text': 'plan: initiate pos', 'event': ['Unk...",0004,26851_333811_0004,[Unknown],plan: initiate pos,0.002435,[],[],[]


In [10]:
N = 10
from config import event_types
import os
os.makedirs("../exports/groundtruth/P-SET/Generated", exist_ok=True)
for ET in event_types:
    top_ET = df_dictionary_exploded[df_dictionary_exploded.Event_Name==ET][["SUBJECT_ID","Event_Name"]].value_counts().reset_index().iloc[:N]
    selected_patient_sentences = df_dictionary[df_dictionary.SUBJECT_ID.isin(top_ET['SUBJECT_ID'].tolist())]
    total_ET = top_ET[top_ET.Event_Name==ET]['count'].sum()
    num_sentences = len(selected_patient_sentences.UID.unique())
    num_reports = len(selected_patient_sentences.ROW_ID.unique())
    num_patients = len(selected_patient_sentences.SUBJECT_ID.unique())
    print(f"***{ET}***",
          f"{ET} counts:{total_ET}\n",
        # f"Event counts:\n{selected_patient_sentences.Event_Name.value_counts()}\n"
        f"Unique Sentences: {num_sentences}\n",
        f"Unique Reports: {num_reports}\n",
        f"Unique Patients: {num_patients}\n"
    )
    selected_patient_sentences.to_pickle(f"../exports/groundtruth/P-SET/Generated/{ET}_{num_patients}_{num_reports}_{num_sentences}_{total_ET}.pkl")

***Pain*** Pain counts:422
 Unique Sentences: 5620
 Unique Reports: 225
 Unique Patients: 10

***Sleep*** Sleep counts:164
 Unique Sentences: 4783
 Unique Reports: 210
 Unique Patients: 10

***Excretion*** Excretion counts:229
 Unique Sentences: 7382
 Unique Reports: 317
 Unique Patients: 10

***Eating*** Eating counts:152
 Unique Sentences: 7267
 Unique Reports: 329
 Unique Patients: 10

***Family*** Family counts:536
 Unique Sentences: 7910
 Unique Reports: 328
 Unique Patients: 10



In [38]:
selected_patient_sentences.UID.unique()

array(['1135_1277059_0000', '1135_1277059_0001', '1135_1277059_0002', ...,
       '32291_1671540_0018', '32291_1671540_0019', '32291_1671540_0020'],
      shape=(4783,), dtype=object)